In [ ]:
!pip install pycuda

In [ ]:
%%writefile sumArrayGPU.cu
// CUDA kernel function

extern "C" __global__ void my_cuda_kernel(int *input, int *output, int size) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid < size) {
        output[tid] = input[tid] * 2;  // Example: Double each element
    }
}

// Wrapper function to call the CUDA kernel
extern "C" void my_cuda_function(int *input, int *output, int size) {
    // No need for memory allocations or movements here.
    // These will be handled by PyCUDA in the Python code.

    // Call the CUDA kernel
    int threadsPerBlock = 256;
    int blocksPerGrid = (size + threadsPerBlock - 1) / threadsPerBlock;
    my_cuda_kernel<<<blocksPerGrid, threadsPerBlock>>>(input, output, size);
}

Overwriting sumArrayGPU.cu


In [ ]:
!nvcc -arch=sm_75 -o sumArrayGPU.so -shared -Xcompiler -fPIC sumArrayGPU.cu

In [ ]:
import ctypes
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import pycuda.autoinit

import numpy as np

# Load the CUDA library
cuda_lib = ctypes.CDLL('./sumArrayGPU.so')  # Update with the correct path

# Define the function prototype
cuda_lib.my_cuda_function.argtypes = [ctypes.POINTER(ctypes.c_int), ctypes.POINTER(ctypes.c_int), ctypes.c_int]
cuda_lib.my_cuda_function.restype = None

# Prepare data
input_data = np.array([3, 1, 33, -4]).astype(np.int32)
output_data = np.array([0, 0, 0, 0]).astype(np.int32)
size = len(input_data)

# Use PyCUDA to allocate GPU memory
input_gpu   = gpuarray.to_gpu(input_data)
output_gpu  = gpuarray.to_gpu(output_data)

# Call the CUDA function
cuda_lib.my_cuda_function(ctypes.cast(input_gpu.ptr, ctypes.POINTER(ctypes.c_int)), ctypes.cast(output_gpu.ptr, ctypes.POINTER(ctypes.c_int)), size)

print(input_gpu)
print(output_gpu)

[ 3  1 33 -4]
[ 6  2 66 -8]
